In [1]:
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

plt.style.use('seaborn-darkgrid')

/var/folders/cy/f8598vm53cggn74m01d5s0rc0000gr/T/ipykernel_32735/3691202613.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [3]:
from statsmodels.tsa.ardl import ardl_select_order


In [4]:
# input_dir = '../../data/final'

# df = pd.read_csv(os.path.join(input_dir, 'df_final_geo_518_2.csv'))

df = pd.read_csv("/Users/yiyi/Desktop/df_final_geo_518.csv")

/var/folders/cy/f8598vm53cggn74m01d5s0rc0000gr/T/ipykernel_32735/240449595.py:5: DtypeWarning: Columns (7,8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/yiyi/Desktop/df_final_geo_518.csv")


In [ ]:
# describe df
df.info()

In [5]:
df.columns

Index(['Unnamed: 0', 'index_x', 'datetime', 'Location name_x', 'ESMI_ID',
       'District', 'State', 'Category', 'Connection Type_x', 'Lat', 'Lon',
       't2m', 'u10', 'v10', 'tp', 'date', 'time', 'station_id', 'index_y',
       'pct_blackout', 'wind_speed', 'Connection Type_y', 'climate_zone_DN',
       'climate_zone_name', 'climate_zone_code', 'tree2000', 'gain',
       'lossyear', 'geometry', 'wind_forest_cover', 'climate_zone_code_2_cat',
       'climate_zone_code_2_Am', 'climate_zone_code_2_Aw',
       'climate_zone_code_2_BSh', 'climate_zone_code_2_Cw'],
      dtype='object')

In [ ]:

df['station_id'].value_counts()

In [6]:

df_377 = df[df['station_id'] == 377]
df_377['datetime'] = pd.to_datetime(df_377['datetime'])


/var/folders/cy/f8598vm53cggn74m01d5s0rc0000gr/T/ipykernel_32735/1947902021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_377['datetime'] = pd.to_datetime(df_377['datetime'])


In [7]:

df_377.head(5)

,Unnamed: 0,index_x,datetime,Location name_x,ESMI_ID,District,State,Category,Connection Type_x,Lat,Lon,t2m,u10,v10,tp,date,time,station_id,index_y,pct_blackout,wind_speed,Connection Type_y,climate_zone_DN,climate_zone_name,climate_zone_code,tree2000,gain,lossyear,geometry,wind_forest_cover,climate_zone_code_2_cat,climate_zone_code_2_Am,climate_zone_code_2_Aw,climate_zone_code_2_BSh,climate_zone_code_2_Cw
5293406,5301279,47724475,2018-06-09 14:00:00,Rajendra Gunj Raichur,NaN,Raichur,Karnataka,District Headquarters,Domestic,16.230193,77.360334,299.74023,4.610990,0.059518,1.814374e-02,20180609,1400,377,2032078,0.0,4.611374,Domestic,6.0,"Arid, steppe, hot",BSh,0.0,0.0,2000.0,POINT (77.36033372 16.2301925),0.0,2,0,0,1,0
5293407,5301280,47724476,2018-11-29 12:00:00,Rajendra Gunj Raichur,NaN,Raichur,Karnataka,District Headquarters,Domestic,16.230193,77.360334,302.49740,-1.692687,-1.023727,1.287539e-06,20181129,1200,377,2034433,0.0,1.978183,Domestic,6.0,"Arid, steppe, hot",BSh,0.0,0.0,2000.0,POINT (77.36033372 16.2301925),0.0,2,0,0,1,0
5293408,5301281,47724477,2018-10-08 00:00:00,Rajendra Gunj Raichur,NaN,Raichur,Karnataka,District Headquarters,Domestic,16.230193,77.360334,297.13970,-1.477317,0.405896,5.705208e-05,20181008,0,377,2033320,0.0,1.532063,Domestic,6.0,"Arid, steppe, hot",BSh,0.0,0.0,2000.0,POINT (77.36033372 16.2301925),0.0,2,0,0,1,0
5293409,5301282,47724478,2018-02-28 21:00:00,Rajendra Gunj Raichur,NaN,Raichur,Karnataka,District Headquarters,Domestic,16.230193,77.360334,294.95615,-1.414260,1.772476,8.531588e-07,20180228,2100,377,2030585,0.0,2.267554,Domestic,6.0,"Arid, steppe, hot",BSh,0.0,0.0,2000.0,POINT (77.36033372 16.2301925),0.0,2,0,0,1,0
5293410,5301283,47724479,2018-07-28 06:00:00,Rajendra Gunj Raichur,NaN,Raichur,Karnataka,District Headquarters,Domestic,16.230193,77.360334,302.63280,6.821611,-0.158919,2.614930e-04,20180728,600,377,2033246,0.0,6.823461,Domestic,6.0,"Arid, steppe, hot",BSh,0.0,0.0,2000.0,POINT (77.36033372 16.2301925),0.0,2,0,0,1,0


In [10]:

def testMaxLagAndTime(df, maxlag):

    '''
    This function is used to test max lag selection for ARDL model and record time.
    
    Args:
    df: pd.DataFrame, the dataframe with a cerntain station_id
    maxlag: int, the maximum lag to be tested
    
    Params:
    See ardl_select_order's parameters
    
    Returns:
    test_model: ARDL model, the model with the best maxlag
    
    '''
    
    print('Begin testing...')
    print(f'testing maxlag: {maxlag}')
    time_start = pd.Timestamp.now()
    
    order_selected= ardl_select_order(df['pct_blackout'],	exog=df[[
                                   't2m', 'wind_speed', 'tp']], maxlag=maxlag, ic='aic', maxorder=maxlag).model.fit()
    
    time_selected = pd.Timestamp.now()
    print(f'time elapsed for selecting order: {time_selected-time_start}')
    
    test_model = order_selected.model.fit()
    time_fit = pd.Timestamp.now()
    print(f'time elapsed for fitting model: {time_fit-time_selected}')
    
    time_end = pd.Timestamp.now()
    print(f'time elapsed for the whole process: {time_end-time_start}')
    print('End testing...')
    print('model summary:')
    print(test_model.summary())
    
    return test_model

In [11]:
test_model = testMaxLagAndTime(df_377, 10)
# takes 30 seconds

Begin testing...
testing maxlag: 10


/Applications/anaconda3/envs/india0/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:01:34.180900
time elapsed for fitting model: 0 days 00:00:00.016849
time elapsed for the whole process: 0 days 00:01:34.197780
End testing...
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                 5410
Model:                  ARDL(2, 0, 5)   Log Likelihood                 281.921
Method:               Conditional MLE   S.D. of innovations              0.230
Date:                Wed, 11 Dec 2024   AIC                           -541.843
Time:                        17:24:49   BIC                           -469.291
Sample:                             5   HQIC                          -516.515
                                 5410                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------

/Applications/anaconda3/envs/india0/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


In [ ]:
test_model = testMaxLagAndTime(df_377, 24)
# takes 15 min

In [37]:
test_model = testMaxLagAndTime(df_377, 17)

testing maxlag: 17
Begin testing


/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:08:24.925978
time elapsed for fitting model: 0 days 00:00:00.003843
time elapsed for the whole process: 0 days 00:08:24.929960
End testing
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                 5410
Model:                  ARDL(2, 0, 5)   Log Likelihood                 281.921
Method:               Conditional MLE   S.D. of innovations              0.230
Date:                Wed, 11 Dec 2024   AIC                           -541.843
Time:                        13:13:13   BIC                           -469.291
Sample:                             5   HQIC                          -516.515
                                 5410                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
co

In [ ]:
df_125 = df[df['station_id'] == 232]
df_232['datetime'] = pd.to_datetime(df_125['datetime'])

/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_4100/1340261838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_125['datetime'] = pd.to_datetime(df_125['datetime'])


In [46]:
testMaxLagAndTime(df_125, 10)

testing maxlag: 10
Begin testing


/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:01:04.267100
time elapsed for fitting model: 0 days 00:00:00.005279
time elapsed for the whole process: 0 days 00:01:04.272485
End testing
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                 6551
Model:              ARDL(10, 0, 2, 0)   Log Likelihood               -1584.916
Method:               Conditional MLE   S.D. of innovations              0.308
Date:                Wed, 11 Dec 2024   AIC                           3203.832
Time:                        13:34:40   BIC                           3319.191
Sample:                            10   HQIC                          3243.719
                                 6551                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------


In [ ]:
df_232 = df[df['station_id'] == 232]
df_232.info()


In [52]:
testMaxLagAndTime(df_232, 10)

testing maxlag: 10
Begin testing


/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:04:18.992568
time elapsed for fitting model: 0 days 00:00:00.016229
time elapsed for the whole process: 0 days 00:04:19.008974
End testing
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                21117
Model:              ARDL(10, 0, 1, 0)   Log Likelihood               -7997.989
Method:               Conditional MLE   S.D. of innovations              0.353
Date:                Wed, 11 Dec 2024   AIC                          16027.978
Time:                        13:47:07   BIC                          16155.296
Sample:                            10   HQIC                         16069.524
                                21117                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
